# Lab 2: Detecting Bad Sensors in Power System Monitoring

In this lab, our goal is to detect bad sensor data measured on the IEEE 14 bus test
system shown below. The power flow equations that couple the voltages and power flows are 
nonlinear in nature, as discussed in class. We will load the sensor data from the
file 'sensorData14Bus.csv', and utilize SVM to perform the bad data detection.
We aim to understand how various parameters such as the nature of the corrupt data,
the number of corrupt data, etc., affect our abilities to classify the data.

<img src="IEEE14bus.png">

### First, we need to call the needed libraries

In [1]:
import numpy as np
import os
from sklearn import preprocessing, svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from IPython.display import Image
import pandas as pd

### Loading the data 

Load the sensor data from the IEEE 14 bus test system, that has 14 buses
 and 20 branches. The data has been generated by adding a small noise
 to feasible voltages and power flows.
     
     Columns 1-14 contain bus voltage magnitudes.
     
     Columns 15-28 contain bus voltage phase angles.
     
     Columns 29-48 contain real power flow on all branches.
     
     Columns 49-68 contain reactive power flow on all branches.

In [2]:
nBuses = 14
nBranches = 20

# Select the bus numbers you monitor. For convenience, we have selected it for you.
# The '-1' makes them columns as per Python's convention of starting to number
# from 0.
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

# Select the branches that you monitor.
branchesToSample = np.array([1, 3, 5, 10, 11, 15, 17, 20]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

# Load the sensor data from the file 'sensorData14Bus.csv' in 'X' from the columns
# specified in 'columnsForBuses' and 'columnsForBranches'. The csv file is comma
# separated. Read a maximum of 5000 lines. Make sure your data is a numpy array
# with each column typecast as 'np.float32'.
X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]

print("Loaded sensor data on IEEE 14 bus system.")
print("Number of data points = %d, number of features = %d"
      % (nDataPoints, nFeatures))

Loaded sensor data on IEEE 14 bus system.
Number of data points = 5000, number of features = 26


### Corruption Models 

Intentionally corrupt the first 'nCorrupt' rows of the data by adding
 a quantity to one or two sensor measurements that is not representative of
 our error model. We aim to study what nature of corruption is easier
 or difficult to detect.
 Specifically, we shall study 3 different models:
 
     1. 'corruptionModel' = 1 : Add a random number with a bias to one of the measurements.
     
     2. 'corruptionModel' = 2 : Add a random number without bias to one of the measurements.
     
     3. 'corruptionModel' = 3 : Add a random number with a bias to both the measurements.
     
In all these cases, we will multiply the sensor data by either a uniform or a normal random number multiplied by 'multiplicationFactor'.


In [3]:
# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 1
multiplicationFactor = 0.5

# Choose which data to tamper with, that can be a voltage magnitude,
# voltage phase angle, real power flow on a branch, reactive power flow
# on a branch. We create functions to extract the relevant column to
# corrupt the corresponding data in the 'ii'-th bus or branch.

voltageMagnitudeColumn = lambda ii: 2*ii

voltageAngleColumn = lambda ii: 2*ii + 1

realPowerColumn = lambda ii: 2 * ii \
                             + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: 1 + 2 * ii \
                             + 2*np.shape(busesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 2

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())


 It is always a good practice to scale your data to run SVM. Notice that we are
 cheating a little when we scale the entire data set 'X', because our training and
 test sets are derived from 'X'. Ideally, one would have to scale the training
 and test sets separately. Create the appropriate labels and shuffle the lists 'X' and 'Y' together.


In [4]:

X = preprocessing.StandardScaler().fit_transform(X)

# Create the labels as a column of 1's for the first 'nCorrupt' rows, and
# 0's for the rest.
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))


# Shuffle the features and the labels together.
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)


Recall from the first lab that 'test_size' determines what fraction of the data becomes your test set.

## Task 1 (10 points)

Split the dataset into two parts: training and testing.
Store the training set in the variables 'trainX' and 'trainY'.
 Store the testing set in the variables 'testX' and 'testY.
 Reserve 20% of the data for testing.
The function 'train_test_split' may prove useful.


In [5]:
# Enter your code here
trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.2)

## Task 2 (10 points)

 Define the support vector machine classifier and train on the variables 'trainX' and 'trainY'. Use the SVC library from sklearn.svm. Only specify three hyper-parameters: 'kernel', 'degree', and 'max_iter'. Limit the maximum number of iterations to 100000 at the most. Set the kernel to be a linear classifier first. You may have to change it to report the results with other kernels. The parameter 'degree' specifies the degree for polynomial kernels. This parameter is not used for other kernels. The functions 'svm.SVC' and 'fit' will prove useful.



In [6]:
# Enter your code here
from sklearn.svm import SVC
clf = SVC(kernel='linear', degree=1, max_iter= 100000)
clf.fit(X, Y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=1, gamma='auto_deprecated',
    kernel='linear', max_iter=100000, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

## Task 3 (10 points)

Predict the labels on the 'testX' dataset and store them in 'predictY'.


In [7]:
# Enter your code here
predictY = clf.predict(testX)

## Task 4 (10 points)

Print the 'classification_report' to see how well 'predictY' matches with 'testY'.


In [8]:
# Enter your code here
print(classification_report(testY, predictY))

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       659
         1.0       1.00      0.91      0.95       341

    accuracy                           0.97      1000
   macro avg       0.98      0.95      0.96      1000
weighted avg       0.97      0.97      0.97      1000



Print svm's internal accuracy score as a percentage.

In [9]:
# Enter your code here
print("Accuracy score of SVM = %1.2f" % clf.score(testX, testY))

Accuracy score of SVM = 0.97


## Task 5

We would like to compare 'classification_report' with this score for various runs. Let us consider the following cases: 

### Case 1:

Only have sensor measurements from the first 5 branches. Choose option 1 in the 'columnsToCorruptOption'. Examine how well linear kernels perform when 'corruptionModel' = 1, 'corruptionModel' = 2, and 'corruptionModel'= 3. In case linear kernels do not perform well, you may try 'rbf' or polynomial kernels with degree 2.

### Case 2:

Choose 'corruptionModel = 1' with 'linear' kernel. Does it pay to monitor voltage magnitudes than power flows? In other words, do you consistently get better results when you choose 'columnsToCorruptOption' as 2? Make these judgements using the average score of at least 5 runs.


#### Your task is to investigate the above two cases. You may add a few 'Markdown' and 'Code' cells below with your comments, code, and results. You can also report your results as a pandas DataFrame. You are free to report your results in your own way.

**PLEASE READ**
* I have outputted all parameters in the code as print statements so it might be better to read just the output of all the cells
* The first few cells are for Case 1, Last 2 cells are for case 2
* All the markdown files with descriptions are at the **bottom** of the notebook. They contain no new information except the outputs from the previous cells and my understanding of the problem. This is where I summarize everything from both the cases in task 5 and write my comments.

CASE 1 CODE 

In [10]:
nBuses = 14
nBranches = 20
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

branchesToSample = np.array([1, 2, 3, 4, 5]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]

print("Buses we've sampled are ",busesToSample + 1)
print("")
print("Branches we've sampled are ", branchesToSample + 1)
print("")

# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 1
print("CORRUPTION MODEL IS", corruptionModel)
multiplicationFactor = 0.5

voltageMagnitudeColumn = lambda ii: 2*ii

voltageAngleColumn = lambda ii: 2*ii + 1

realPowerColumn = lambda ii: 2 * ii \
                             + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: 1 + 2 * ii \
                             + 2*np.shape(busesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 1

print("")
print("COLUMNS TO CORRUPT OPTION IS ", columnsToCorruptOption)
print("")

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())
        

X = preprocessing.StandardScaler().fit_transform(X)
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)

trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.2)

from sklearn.svm import SVC
clf = SVC(kernel='linear', degree=2, max_iter= 100000)
clf.fit(X, Y)

predictY = clf.predict(testX)

print(classification_report(testY, predictY))
print("Accuracy score of SVM = %1.2f" % clf.score(testX, testY))

Buses we've sampled are  [ 1  2  5 10 13]

Branches we've sampled are  [1 2 3 4 5]

CORRUPTION MODEL IS 1

COLUMNS TO CORRUPT OPTION IS  1

              precision    recall  f1-score   support

         0.0       0.74      0.96      0.83       671
         1.0       0.78      0.30      0.43       329

    accuracy                           0.74      1000
   macro avg       0.76      0.63      0.63      1000
weighted avg       0.75      0.74      0.70      1000

Accuracy score of SVM = 0.74


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:241: ConvergenceWarning: Solver terminated early (max_iter=100000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


In [11]:
nBuses = 14
nBranches = 20
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

branchesToSample = np.array([1, 2, 3, 4, 5]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]

print("Buses we've sampled are ",busesToSample + 1)
print("")
print("Branches we've sampled are ", branchesToSample + 1)
print("")

# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 2
print("CORRUPTION MODEL IS", corruptionModel)
multiplicationFactor = 0.5

voltageMagnitudeColumn = lambda ii: 2*ii

voltageAngleColumn = lambda ii: 2*ii + 1

realPowerColumn = lambda ii: 2 * ii \
                             + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: 1 + 2 * ii \
                             + 2*np.shape(busesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 1

print("")
print("COLUMNS TO CORRUPT OPTION IS ", columnsToCorruptOption)
print("")

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())
        

X = preprocessing.StandardScaler().fit_transform(X)
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)

trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.2)

from sklearn.svm import SVC
clf = SVC(kernel='linear', degree=1, max_iter= 100000)
clf.fit(X, Y)

predictY = clf.predict(testX)

print(classification_report(testY, predictY))
print("Accuracy score of SVM = %1.2f" % clf.score(testX, testY))
print("Kernel is Linear")

Buses we've sampled are  [ 1  2  5 10 13]

Branches we've sampled are  [1 2 3 4 5]

CORRUPTION MODEL IS 2

COLUMNS TO CORRUPT OPTION IS  1

              precision    recall  f1-score   support

         0.0       0.66      1.00      0.80       662
         1.0       0.00      0.00      0.00       338

    accuracy                           0.66      1000
   macro avg       0.33      0.50      0.40      1000
weighted avg       0.44      0.66      0.53      1000

Accuracy score of SVM = 0.66
Kernel is Linear


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Since the recall and precision of 1.0 are both 0, we will not use a linear kernel and use rbf with gama = auto instead

In [16]:
nBuses = 14
nBranches = 20
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

branchesToSample = np.array([1, 2, 3, 4, 5]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]

print("Buses we've sampled are ",busesToSample + 1)
print("")
print("Branches we've sampled are ", branchesToSample + 1)
print("")

# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 2
print("CORRUPTION MODEL IS", corruptionModel)
multiplicationFactor = 0.5

voltageMagnitudeColumn = lambda ii: 2*ii

voltageAngleColumn = lambda ii: 2*ii + 1

realPowerColumn = lambda ii: 2 * ii \
                             + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: 1 + 2 * ii \
                             + 2*np.shape(busesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 1

print("")
print("COLUMNS TO CORRUPT OPTION IS ", columnsToCorruptOption)
print("")

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())
        

X = preprocessing.StandardScaler().fit_transform(X)
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)

trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.2)

from sklearn.svm import SVC
clf = SVC(kernel='rbf', gamma = 'auto', max_iter= 100000)
clf.fit(X, Y)

predictY = clf.predict(testX)

print(classification_report(testY, predictY))
print("Accuracy score of SVM = %1.2f" % clf.score(testX, testY))
print("Kernel is rbf")

Buses we've sampled are  [ 1  2  5 10 13]

Branches we've sampled are  [1 2 3 4 5]

CORRUPTION MODEL IS 2

COLUMNS TO CORRUPT OPTION IS  1

              precision    recall  f1-score   support

         0.0       0.75      1.00      0.85       681
         1.0       1.00      0.27      0.42       319

    accuracy                           0.77      1000
   macro avg       0.87      0.63      0.64      1000
weighted avg       0.83      0.77      0.72      1000

Accuracy score of SVM = 0.77
Kernel is rbf


Using a linear kernel, with 1 column to corrupt and corruption model 2, we get an accuracy of 0.65 with a weighted precision of 0.43 and recall 0.66 which is too low. Since this was too low, I changed it to an rbf kernel and got a higher accuracy of 0.76 with precision 0.82 and recall of 0.76. 

In [13]:
nBuses = 14
nBranches = 20
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

branchesToSample = np.array([1, 2, 3, 4, 5]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]

print("Buses we've sampled are ",busesToSample + 1)
print("")
print("Branches we've sampled are ", branchesToSample + 1)
print("")

# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 3
print("CORRUPTION MODEL IS", corruptionModel)
multiplicationFactor = 0.5

voltageMagnitudeColumn = lambda ii: 2*ii

voltageAngleColumn = lambda ii: 2*ii + 1

realPowerColumn = lambda ii: 2 * ii \
                             + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: 1 + 2 * ii \
                             + 2*np.shape(busesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 1

print("")
print("COLUMNS TO CORRUPT OPTION IS ", columnsToCorruptOption)
print("")

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())
        

X = preprocessing.StandardScaler().fit_transform(X)
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)

trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.2)

from sklearn.svm import SVC
clf = SVC(kernel='linear', degree=2, max_iter= 100000)
clf.fit(X, Y)

predictY = clf.predict(testX)

print(classification_report(testY, predictY))
print("Accuracy score of SVM = %1.2f" % clf.score(testX, testY))

Buses we've sampled are  [ 1  2  5 10 13]

Branches we've sampled are  [1 2 3 4 5]

CORRUPTION MODEL IS 3

COLUMNS TO CORRUPT OPTION IS  1

              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99       683
         1.0       0.99      0.94      0.97       317

    accuracy                           0.98      1000
   macro avg       0.98      0.97      0.98      1000
weighted avg       0.98      0.98      0.98      1000

Accuracy score of SVM = 0.98


In [14]:
nBuses = 14
nBranches = 20
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

branchesToSample = np.array([1, 2, 3, 4, 5]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]

print("Buses we've sampled are",busesToSample)
print("")
print("Branches we've sampled are", branchesToSample)
print("")

# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 1
print("CORRUPTION MODEL IS", corruptionModel)
multiplicationFactor = 0.5

voltageMagnitudeColumn = lambda ii: 2*ii

voltageAngleColumn = lambda ii: 2*ii + 1

realPowerColumn = lambda ii: 2 * ii \
                             + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: 1 + 2 * ii \
                             + 2*np.shape(busesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 2

print("")
print("COLUMNS TO CORRUPT OPTION IS ", columnsToCorruptOption)
print("")

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())
        

X = preprocessing.StandardScaler().fit_transform(X)
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)

trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.2)

from sklearn.svm import SVC
clf = SVC(kernel='linear', degree=2, max_iter= 100000)
clf.fit(X, Y)

predictY = clf.predict(testX)

print(classification_report(testY, predictY))
print("Accuracy score of SVM = %1.2f" % clf.score(testX, testY))

Buses we've sampled are [ 0  1  4  9 12]

Branches we've sampled are [0 1 2 3 4]

CORRUPTION MODEL IS 1

COLUMNS TO CORRUPT OPTION IS  2

              precision    recall  f1-score   support

         0.0       0.96      1.00      0.98       670
         1.0       1.00      0.92      0.96       330

    accuracy                           0.97      1000
   macro avg       0.98      0.96      0.97      1000
weighted avg       0.97      0.97      0.97      1000

Accuracy score of SVM = 0.97


In [15]:
nBuses = 14
nBranches = 20
busesToSample = np.array([1, 2, 5, 10, 13]) - 1
columnsForBuses = np.concatenate((busesToSample, busesToSample + 14))

branchesToSample = np.array([1, 2, 3, 4, 5]) - 1
columnsForBranches = np.concatenate((branchesToSample + 28,
                                     branchesToSample + 48))

X = np.genfromtxt('sensorData14Bus.csv', dtype=np.float32, delimiter=',',
                  usecols=np.concatenate((columnsForBuses, columnsForBranches)),
                  max_rows=5000)

nDataPoints = np.shape(X)[0]
nFeatures = np.shape(X)[1]

print("Buses we've sampled are",busesToSample)
print("")
print("Branches we've sampled are ", branchesToSample)
print("")

# Choose a corruption model.
nCorrupt = int(nDataPoints/3)
corruptionModel = 1
print("CORRUPTION MODEL IS", corruptionModel)
multiplicationFactor = 0.5

voltageMagnitudeColumn = lambda ii: 2*ii

voltageAngleColumn = lambda ii: 2*ii + 1

realPowerColumn = lambda ii: 2 * ii \
                             + 2*np.shape(busesToSample)[0]
reactivePowerColumn = lambda ii: 1 + 2 * ii \
                             + 2*np.shape(busesToSample)[0]

# Encode two different kinds of columns to corrupt.
# Option 1: Corrupt real power columns only.
# Option 2: Corrupt real power and voltage magnitude.
columnsToCorruptOption = 1

print("")
print("COLUMNS TO CORRUPT OPTION IS ", columnsToCorruptOption)
print("")

if columnsToCorruptOption == 1:
    columnsToCorrupt = [realPowerColumn(1),
                        realPowerColumn(2)]
else:
    columnsToCorrupt = [voltageMagnitudeColumn(0),
                        realPowerColumn(1)]

# Corrupt the data appropriately, given the options.
for index in range(nCorrupt):

    if corruptionModel == 1:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
    elif corruptionModel == 2:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.randn())
    else:
        X[index, columnsToCorrupt[0]] \
            *= (1 + multiplicationFactor * np.random.rand())
        X[index, columnsToCorrupt[1]] \
            *= (1 + multiplicationFactor * np.random.rand())
        

X = preprocessing.StandardScaler().fit_transform(X)
Y = np.concatenate((np.ones(nCorrupt), np.zeros(nDataPoints-nCorrupt)))
XY = list(zip(X, Y))
np.random.shuffle(XY)
X, Y = zip(*XY)

trainX, testX, trainY, testY = train_test_split(X, Y,test_size=0.2)

from sklearn.svm import SVC
clf = SVC(kernel='linear', degree=2, max_iter= 100000)
clf.fit(X, Y)

predictY = clf.predict(testX)

print(classification_report(testY, predictY))
print("Accuracy score of SVM = %1.2f" % clf.score(testX, testY))

Buses we've sampled are [ 0  1  4  9 12]

Branches we've sampled are  [0 1 2 3 4]

CORRUPTION MODEL IS 1

COLUMNS TO CORRUPT OPTION IS  1

              precision    recall  f1-score   support

         0.0       0.75      0.97      0.84       684
         1.0       0.80      0.30      0.43       316

    accuracy                           0.76      1000
   macro avg       0.78      0.63      0.64      1000
weighted avg       0.77      0.76      0.71      1000

Accuracy score of SVM = 0.76


**CASE 1**

Performance of Kernels with different corruption model with columns to corrupt = 1

| Corrupt 1 Column -> | Model 1* |  Model 2* | Model 2 w/ RBF | Model 3 |
| :---                 |    :----:   |          ---: |          ---: |           ---: |
| Accuracy | 0.71       | 0.65   | 0.77   |  0.97   |
| Weighted F1 Score| 0.66     | 0.52  | 0.71  |  0.97  |


*Models did not converge

* For Models 1 and 2, there was no convergence. As a result, the accuracy was low. This is probably due to the fact that we were adding bias to only one column which did not generalize well for the SVM model. 

* However, for Model 2 with RBF, we were able to see an increase in accuracy from 0.65 to 0.77 and an increase in both recall and precision as well. Since F1 score is just the weighted average of precision and recall, we saw an increase from 0.52 to 0.71 as well. Non-Linearity might have helped us fit the model better and thus caused an increase in the accuracy and f1 score

* Model 3 worked the best since the accuracy and f1 score was the highest (0.97). This is probably because we added bias to both measures and that might have generalized better to the test data. Therefore we know that model 3 is superior to both the other models. 

**CASE 2**

Performance with 1 or 2 columns corrupted. We are using corruption Model 1 for both cases.


| 1 Corrupt Column -> | Run 1 |  Run 2 | Run 3 | Run 4 | Run 5 |/| **Average** |
| :---                 |    :----:   |          ---: |          ---: | ---: | ---:|  ---: |  ---: |
| **Accuracy**             | 0.74        | 0.76          | 0.75          | 0.75 | 0.74 | /| **0.748** |
| **Weighted F1 Score**    | 0.71        | 0.72         | 0.71          | 0.71 | 0.70 |/ | **0.71** |

| 2 Corrupted Columns -> | Run 1 |  Run 2 | Run 3 | Run 4 | Run 5 |/| **Average** |
| :---                 |    :----:   |          ---: |          ---: | ---: | ---:|  ---: |  ---: |
| **Accuracy**             | 0.98        | 0.98         | 0.97          | 0.98 | 0.98 |/ | **0.978** |
| **Weighted F1 Score**    | 0.98        | 0.98          | 0.97          | 0.98 | 0.98 |/| **0.978** |

We see that there is a sufficient increase in average accuracy from 0.748 to 0.978 with the addition of corrupting both the columns. There is an increase in both precision and recall too and as a result, we see an increase in the F1 score as well from (0.71 to 0.978) This is probably due to the fact that adding noise to the data makes the model more generalizable and makes it a more realistic model. Therefore, we see an increase in accuracy for the model. 